# Topics – Easy Topic Modeling in Python

The text mining technique **Topic Modeling** has become a popular statistical method for clustering documents. This [Jupyter notebook](http://jupyter.org/) introduces a step-by-step workflow, basically containing data preprocessing, the actual topic modeling using **latent Dirichlet allocation** (LDA), which learns the relationships between words, topics, and documents, as well as some interactive visualizations to explore the model.

LDA, introduced in the context of text analysis in [2003](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf), is an instance of a more general class of models called **mixed-membership models**. Involving a number of distributions and parameters, the topic model is typically performed using [Gibbs sampling](https://en.wikipedia.org/wiki/Gibbs_sampling) with conjugate priors and is purely based on word frequencies.

There have been written numerous introductions to topic modeling for humanists (e.g. [this one](http://scottbot.net/topic-modeling-for-humanists-a-guided-tour/)), which provide another level of detail regarding its technical and epistemic properties.

For this workflow, you will need a corpus (a set of texts) as plain text (`.txt`) or [TEI XML](http://www.tei-c.org/index.xml) (`.xml`). Using the `dariah_topics` package, you also have the ability to process the output of [DARIAH-DKPro-Wrapper](https://github.com/DARIAH-DE/DARIAH-DKPro-Wrapper), a command-line tool for *natural language processing*.

Topic modeling works best with very large corpora. The [TextGrid Repository](https://textgridrep.org/) is a great place to start searching for text data. Anyway, to demonstrate the technique, we provide one small text collection in the folder `grenzboten_sample` containing 15 diary excerpts, as well as 15 war diary excerpts, which appeared in *Die Grenzboten*, a German newspaper of the late 19th and early 20th century.

**Of course, you can work with your own corpus in this notebook.**

We're relying on the LDA implementation by [Andrew McCallum](https://people.cs.umass.edu/~mccallum/), called [MALLET](http://mallet.cs.umass.edu/topics.php), which is known to be very robust. Aside from that, we provide two more Jupyter notebooks:

* [IntroducingMallet](IntroducingGensim.ipynb), using LDA by [Gensim](https://radimrehurek.com/project/gensim/), which is attractive because of its multi-core support.
* [IntroducingLda](IntroducingLda.ipynb), using LDA by [lda](http://pythonhosted.org/lda/index.html), which is very lightweight.

For more information in general, have a look at the [documentation](http://dev.digital-humanities.de/ci/job/DARIAH-Topics/doclinks/1/).

## First step: Installing dependencies

To work within this Jupyter notebook, you will have to import the `dariah_topics` library. As you do, `dariah_topics` also imports a couple of external libraries, which have to be installed first. `pip` is the preferred installer program in Python. Starting with Python 3.4, it is included by default with the Python binary installers. If you are interested in `pip`, have a look at [this website](https://docs.python.org/3/installing/index.html).

You have the ability to install dependencies via `pip` from within this notebook. To get a feeling for working with Jupyter, copy and paste (or best: type) the following code snippet in the empty cell below and press the **Run**-button.

```
import pip

pip.main(['install', '-r', 'requirements.txt'])
```

If you get any errors or are not able to install *all* dependencies properly, try [Stack Overflow](https://stackoverflow.com/questions/tagged/pip) for troubleshooting or create a new issue on our [GitHub page](https://github.com/DARIAH-DE/Topics).

**Furthermore**, you will have to install MALLET, or at least [download](http://mallet.cs.umass.edu/download.php), unzip and specifiy the path to the unzipped folder further below.

### What have I done?

With `import pip` you have imported the package `pip`, which is in the Python standard library included. In the next line, you called `pip`'s function `main` and commited a [list](https://en.wikipedia.org/wiki/List_(abstract_data_type) with three elements:

1. `install` is the command to install packages.
2. `-r` (or `--requirements`) installs from the given requirements file.
3. `requirements.txt` is a simple text file containing a list of all required libraries.

### Some final words
As you already know, code has to be written in the grey cells. You execute a cell by clicking the **Run**-button. If you want to run all cells of the notebook at once, click **Cell > Run All** or **Kernel > Restart & Run All** respectively, if you want to restart the Python kernel first. On the left side of an (unexecuted) cell stands `In [ ]:`. The empty bracket means, that the cell hasn't been executed yet. By clicking **Run**, a star appears in the brackets (`In [*]:`), which means the process is running. In most cases, you won't see that star, because your computer is faster than your eyes. You can execute only one cell at once, all following executions will be in the waiting line. If the process of a cell is done, a number appears in the brackets (`In [1]:`).

## Starting with topic modeling!

Execute the following cell to import modules from the `dariah_topics` library.

In [1]:
from dariah_topics import preprocessing
from dariah_topics import doclist
from dariah_topics import meta
from dariah_topics import mallet
from dariah_topics import visualization

Let's not pay heed to any warnings right now and execute the following cell.

In [2]:
import warnings
warnings.filterwarnings('ignore')

## 1. Preprocessing

### 1.1. Reading a corpus of documents

#### Defining the path to the corpus folder

In the present example code, we are using a folder of 'txt' documents provided with the package. For using your own corpus, change the path accordingly.

In [3]:
path = "grenzboten_sample"

#### List all documents in the folder
We begin by creating a list of all the documents in the folder specified above. That list will tell function `pre.read_from_txt()` (see below) which text documents to read.

In [4]:
pathdoclist = doclist.PathDocList(path)
document_list = pathdoclist.full_paths(as_str=True)

The current list of documents looks like this:

In [5]:
document_list

['grenzboten_sample/Grenzboten_1844_Tagebuch_56.txt',
 'grenzboten_sample/Grenzboten_1846_Tagebuch_82.txt',
 'grenzboten_sample/Grenzboten_1916_Kriegstagebuch_69.txt',
 'grenzboten_sample/Grenzboten_1915_Kriegstagebuch_73.txt',
 'grenzboten_sample/Grenzboten_1914_Kriegstagebuch_95.txt',
 'grenzboten_sample/Grenzboten_1915_Kriegstagebuch_33.txt',
 'grenzboten_sample/Grenzboten_1914_Kriegstagebuch_68.txt',
 'grenzboten_sample/Grenzboten_1846_Tagebuch_51.txt',
 'grenzboten_sample/Grenzboten_1845_Tagebuch_81.txt',
 'grenzboten_sample/Grenzboten_1844_Tagebuch_82.txt',
 'grenzboten_sample/Grenzboten_1916_Kriegstagebuch_48.txt',
 'grenzboten_sample/Grenzboten_1915_Kriegstagebuch_94.txt',
 'grenzboten_sample/Grenzboten_1915_Kriegstagebuch_39.txt',
 'grenzboten_sample/Grenzboten_1845_Tagebuch_85.txt',
 'grenzboten_sample/Grenzboten_1846_Tagebuch_96.txt',
 'grenzboten_sample/Grenzboten_1845_Tagebuch_93.txt',
 'grenzboten_sample/Grenzboten_1916_Kriegstagebuch_81.txt',
 'grenzboten_sample/Grenzbot

**Alternatively**, if we want to use other documents, or just a selction of those in the specified folder, we can define our own `doclist` by creating a list of strings containing paths to text files. For example, to use only the texts from 1916, we would define the list as

`
    doclist = ['grenzboten_sample/grenzboten_1916_Kriegstagebuch_41.txt',
           'grenzboten_sample/grenzboten_1916_Kriegstagebuch_48.txt',
           'grenzboten_sample/grenzboten_1916_Kriegstagebuch_49.txt',
           'grenzboten_sample/grenzboten_1916_Kriegstagebuch_69.txt',
           'grenzboten_sample/grenzboten_1916_Kriegstagebuch_81.txt']
`

#### Generate document labels

In [6]:
document_labels = pathdoclist.labels()
document_labels

['Grenzboten_1844_Tagebuch_56',
 'Grenzboten_1846_Tagebuch_82',
 'Grenzboten_1916_Kriegstagebuch_69',
 'Grenzboten_1915_Kriegstagebuch_73',
 'Grenzboten_1914_Kriegstagebuch_95',
 'Grenzboten_1915_Kriegstagebuch_33',
 'Grenzboten_1914_Kriegstagebuch_68',
 'Grenzboten_1846_Tagebuch_51',
 'Grenzboten_1845_Tagebuch_81',
 'Grenzboten_1844_Tagebuch_82',
 'Grenzboten_1916_Kriegstagebuch_48',
 'Grenzboten_1915_Kriegstagebuch_94',
 'Grenzboten_1915_Kriegstagebuch_39',
 'Grenzboten_1845_Tagebuch_85',
 'Grenzboten_1846_Tagebuch_96',
 'Grenzboten_1845_Tagebuch_93',
 'Grenzboten_1916_Kriegstagebuch_81',
 'Grenzboten_1845_Tagebuch_62',
 'Grenzboten_1844_Tagebuch_77',
 'Grenzboten_1914_Kriegstagebuch_97',
 'Grenzboten_1916_Kriegstagebuch_41',
 'Grenzboten_1916_Kriegstagebuch_49',
 'Grenzboten_1844_Tagebuch_70',
 'Grenzboten_1914_Kriegstagebuch_37',
 'Grenzboten_1844_Tagebuch_88',
 'Grenzboten_1845_Tagebuch_52',
 'Grenzboten_1915_Kriegstagebuch_99',
 'Grenzboten_1914_Kriegstagebuch_94',
 'Grenzboten_1

#### Optional: Accessing metadata

In case you want a more structured overview of your corpus, execute the following cell:

In [7]:
import os

metadata = meta.fn2metadata(os.path.join(path, '*.txt'))
metadata

,author,basename,filename,title
0,Grenzboten,Grenzboten_1844_Tagebuch_56,grenzboten_sample/Grenzboten_1844_Tagebuch_56.txt,1844_Tagebuch_56
1,Grenzboten,Grenzboten_1846_Tagebuch_82,grenzboten_sample/Grenzboten_1846_Tagebuch_82.txt,1846_Tagebuch_82
2,Grenzboten,Grenzboten_1916_Kriegstagebuch_69,grenzboten_sample/Grenzboten_1916_Kriegstagebu...,1916_Kriegstagebuch_69
3,Grenzboten,Grenzboten_1915_Kriegstagebuch_73,grenzboten_sample/Grenzboten_1915_Kriegstagebu...,1915_Kriegstagebuch_73
4,Grenzboten,Grenzboten_1914_Kriegstagebuch_95,grenzboten_sample/Grenzboten_1914_Kriegstagebu...,1914_Kriegstagebuch_95
5,Grenzboten,Grenzboten_1915_Kriegstagebuch_33,grenzboten_sample/Grenzboten_1915_Kriegstagebu...,1915_Kriegstagebuch_33
6,Grenzboten,Grenzboten_1914_Kriegstagebuch_68,grenzboten_sample/Grenzboten_1914_Kriegstagebu...,1914_Kriegstagebuch_68
7,Grenzboten,Grenzboten_1846_Tagebuch_51,grenzboten_sample/Grenzboten_1846_Tagebuch_51.txt,1846_Tagebuch_51
8,Grenzboten,Grenzboten_1845_Tagebuch_81,grenzboten_sample/Grenzboten_1845_Tagebuch_81.txt,1845_Tagebuch_81
9,Grenzboten,Grenzboten_1844_Tagebuch_82,grenzboten_sample/Grenzboten_1844_Tagebuch_82.txt,1844_Tagebuch_82


#### Read listed documents from folder

In [8]:
corpus = preprocessing.read_from_txt(document_list)

At this point, the corpus is generator object.

### 1.3. Tokenize corpus
Your text files will be tokenized. Tokenization is the task of cutting a stream of characters into linguistic units, simply words or, more precisely, tokens. The tokenize function the library provides is a simple unicode tokenizer. Depending on the corpus it might be useful to use an external tokenizer function, or even develop your own, since its efficiency varies with language, epoch and text type.

In [9]:
tokens = [list(preprocessing.tokenize(document)) for document in list(corpus)]

At this point, each text is represented by a list of separate token strings. If we want to look e.g. into the first text (which has the index `0` as Python starts counting at 0) and show its first 10 words/tokens (that have the indeces `0:9` accordingly) by typing:

In [10]:
tokens[0][0:9]

['es',
 'berlin',
 'und',
 'paris',
 'sprcchscligkeir',
 'credit',
 'und',
 'religion',
 'priester']

### 1.4.1 Create a document-term matrix

The LDA topic model is based on a [document-term matrix](https://en.wikipedia.org/wiki/Document-term_matrix) of the corpus. To improve performance in large corpora, the matrix describes the frequency of terms that occur in the collection. In a document-term matrix, rows correspond to documents in the collection and columns correspond to terms.

In [11]:
doc_terms = preprocessing.create_doc_term_matrix(tokens, document_labels)
doc_terms

,die,der,und,in,den,von,zu,das,des,nicht,...,mördern,mühevolle,münch-bellinghausen,mühling,mühsame,mühsamen,müht,mül,müllers,a!s
Grenzboten_1844_Tagebuch_56,90.0,92.0,88.0,70.0,30.0,25.0,25.0,16.0,25.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1846_Tagebuch_82,319.0,346.0,275.0,164.0,106.0,87.0,110.0,94.0,75.0,96.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Grenzboten_1916_Kriegstagebuch_69,39.0,64.0,51.0,24.0,14.0,28.0,1.0,7.0,10.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Grenzboten_1915_Kriegstagebuch_73,41.0,51.0,43.0,31.0,27.0,7.0,1.0,7.0,9.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1914_Kriegstagebuch_95,80.0,85.0,62.0,65.0,42.0,35.0,11.0,13.0,14.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1915_Kriegstagebuch_33,93.0,95.0,87.0,78.0,50.0,48.0,1.0,8.0,21.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1914_Kriegstagebuch_68,32.0,31.0,26.0,37.0,15.0,13.0,2.0,10.0,16.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1846_Tagebuch_51,226.0,177.0,188.0,111.0,73.0,62.0,93.0,60.0,35.0,78.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1845_Tagebuch_81,344.0,351.0,311.0,178.0,107.0,118.0,156.0,116.0,91.0,112.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1844_Tagebuch_82,213.0,207.0,169.0,128.0,85.0,86.0,79.0,80.0,66.0,67.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.4.2 Create a sparse bag-of-words model

The LDA topic model is based on a bag-of-words model of the corpus. To improve performance in large corpora, actual words and document titels are replaced by indices in the actual bag-of-words model. It is therefore necessary to create dictionaries for mapping these indices in advance.

#### Create Dictionaries

In [12]:
id_types = preprocessing.create_dictionary(tokens)
doc_ids = preprocessing.create_dictionary(document_labels)

#### Create matrix market

In [13]:
sparse_bow = preprocessing.create_sparse_bow(document_labels, tokens, id_types, doc_ids)

### 1.5. Feature selection and/or removal

In topic modeling, it is often usefull (if not vital) to remove some types before modeling. In this example, the 100 most frequent words and the *hapax legomena* in the corpus are listed and removed. Alternatively, the 'feature_list' containing all features to be removed from the corpus can be replaced by, or combined with an external stop word list or any other list of strings containing features we want to remove.

**Note**: For small/normal corpora using a **`doc_term_matrix` (1.5.1)** will be easier to handle. Using **`sparse_bow` (1.5.2)** is recommended for large corpora only.

### 1.5.1 Remove features from `doc_term_matrix`

#### List the 100 most frequent words

In [14]:
mfw100 = preprocessing.find_stopwords(doc_terms, 100)

These are the five most frequent words:

In [15]:
mfw100[:5]

['die', 'der', 'und', 'in', 'den']

#### List hapax legomena

In [16]:
hapax_list = preprocessing.find_hapax(doc_terms)

#### Optional: Use external stopwordlist

In [17]:
path_to_stopwordlist = "tutorial_supplementals/stopwords/de.txt"

extern_stopwords = [line.strip() for line in open(path_to_stopwordlist, 'r')]

#### Combine lists and remove content from `doc_term_matrix`

In [18]:
features = set(mfw100 + hapax_list + extern_stopwords)
doc_terms = preprocessing.remove_features_from_df(doc_terms, features)

Finally, this is how your clean corpus looks like now.

In [19]:
doc_terms

,franzosen,genommen,abgewiesen,südlich,berlin,lassen,geschütze,englische,januar,deutschland,...,tilemans,tausendmal,taten,geldinstitute,tatkraft,gemeinem,tausenden,teilangriffe,tendenzstück,gemeingefährlichkeitsmaßstab
Grenzboten_1844_Tagebuch_56,0.0,1.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1846_Tagebuch_82,4.0,2.0,0.0,0.0,1.0,7.0,0.0,1.0,1.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1916_Kriegstagebuch_69,12.0,6.0,9.0,10.0,2.0,0.0,5.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1915_Kriegstagebuch_73,8.0,11.0,6.0,7.0,2.0,0.0,6.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1914_Kriegstagebuch_95,2.0,9.0,12.0,6.0,2.0,1.0,8.0,10.0,1.0,1.0,...,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1915_Kriegstagebuch_33,17.0,13.0,24.0,8.0,4.0,1.0,16.0,11.0,86.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1914_Kriegstagebuch_68,8.0,3.0,5.0,3.0,2.0,0.0,3.0,3.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1846_Tagebuch_51,1.0,1.0,1.0,0.0,1.0,7.0,0.0,1.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Grenzboten_1845_Tagebuch_81,1.0,3.0,0.0,0.0,7.0,10.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0
Grenzboten_1844_Tagebuch_82,0.0,0.0,0.0,0.0,5.0,10.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.5.2 Remove features from `sparse_bow`

#### List the 100 most frequent words

In [20]:
mfw100 = preprocessing.find_stopwords(sparse_bow, 100, id_types)

These are the five most frequent words:

In [21]:
mfw100[:5]

['die', 'der', 'und', 'in', 'den']

#### List hapax legomena

In [22]:
hapax_list = preprocessing.find_hapax(sparse_bow, id_types)

#### Optional: Use external stopwordlist

In [23]:
path_to_stopwordlist = "tutorial_supplementals/stopwords/de.txt"

extern_stopwords = [line.strip() for line in open(path_to_stopwordlist, 'r')]

#### Combine lists

In [24]:
features = set(mfw100 + hapax_list + extern_stopwords)

#### Remove features from files

In [25]:
tokens_cleaned = []

for document in tokens:
    document_clean = preprocessing.remove_features_from_file(document, list(features))
    tokens_cleaned.append(list(document_clean))

#### Write MALLET import files

In [26]:
preprocessing.create_mallet_import(tokens_cleaned, document_labels)

## 1. Setting the parameters

#### Define path to corpus folder

In [27]:
path_to_corpus = "tutorial_supplementals/mallet_input"

#### Path to mallet folder 

Now we must tell the library where to find the local instance of mallet. If you managed to install Mallet, it is sufficient set `path_to_mallet = "mallet"`, if you just store Mallet in a local folder, you have to specify the path to the binary explictly.

In [28]:
path_to_mallet = 'mallet'

#### Output folder

In [29]:
outfolder = "tutorial_supplementals/mallet_output"
binary = "tutorial_supplementals/mallet_output/binary.mallet"

#### The Mallet corpus model

Finally, we can give all these folder paths to a Mallet function that handles all the preprocessing steps and creates a Mallet-specific corpus model object.

In [30]:
%%time

mallet_binary = mallet.create_mallet_binary(path_to_mallet=path_to_mallet,
                                            path_to_corpus=path_to_corpus,
                                            output_file=binary) 

CPU times: user 0 ns, sys: 7.45 ms, total: 7.45 ms
Wall time: 271 ms


## 2. Model creation

We can define the number of topics we want to calculate as an argument (`num_topics`) in the function. Furthermore, the number of iterations (`num_iterations`) can be defined. A higher number of iterations will probably yield a better model, but also increases processing time.

**Warning: this step can take quite a while!** Meaning something between some seconds and some hours depending on corpus size and the number of iterations. Our example short stories corpus should be done within a minute or two at `num_iterations=5000`.

In [31]:
%%time

mallet.create_mallet_model(path_to_mallet=path_to_mallet, 
                           path_to_binary=mallet_binary, 
                           folder_for_output=outfolder,
                           num_iterations=5000,
                           num_topics=10)

CPU times: user 4.98 ms, sys: 3.73 ms, total: 8.72 ms
Wall time: 412 ms


### 2.4. Create document-topic matrix

The generated model object can now be translated into a human-readable document-topic matrix (that is a actually a pandas data frame) that constitutes our principle exchange format for topic modeling results. For generating the matrix from a Gensim model, we can use the following function:

In [32]:
doc_topics = mallet.show_doc_topic_matrix(outfolder)
doc_topics

,Grenzboten_1844_Tagebuch_56.txt,Grenzboten_1844_Tagebuch_70.txt,Grenzboten_1844_Tagebuch_77.txt,Grenzboten_1844_Tagebuch_82.txt,Grenzboten_1844_Tagebuch_88.txt,Grenzboten_1845_Tagebuch_52.txt,Grenzboten_1845_Tagebuch_62.txt,Grenzboten_1845_Tagebuch_81.txt,Grenzboten_1845_Tagebuch_85.txt,Grenzboten_1845_Tagebuch_93.txt,...,Grenzboten_1915_Kriegstagebuch_33.txt,Grenzboten_1915_Kriegstagebuch_39.txt,Grenzboten_1915_Kriegstagebuch_73.txt,Grenzboten_1915_Kriegstagebuch_94.txt,Grenzboten_1915_Kriegstagebuch_99.txt,Grenzboten_1916_Kriegstagebuch_41.txt,Grenzboten_1916_Kriegstagebuch_48.txt,Grenzboten_1916_Kriegstagebuch_49.txt,Grenzboten_1916_Kriegstagebuch_69.txt,Grenzboten_1916_Kriegstagebuch_81.txt
drei september deutscher,0.069915,0.053368,0.047408,0.046118,0.059833,0.070327,0.035205,0.040581,0.033713,0.058813,...,0.040694,0.037059,0.079969,0.061141,0.074966,0.017330,0.019620,0.014646,0.017127,0.045207
leben deutschland kunst,0.058898,0.069603,0.089128,0.049323,0.079314,0.063242,0.066993,0.036465,0.068137,0.243907,...,0.014508,0.010000,0.002329,0.014946,0.002063,0.005777,0.000633,0.005556,0.003867,0.001634
vom einmal wiener,0.137712,0.274093,0.143911,0.099893,0.350186,0.147635,0.057190,0.076703,0.099716,0.041858,...,0.004600,0.002941,0.002329,0.001359,0.002063,0.003209,0.000633,0.000505,0.001657,0.000545
leipzig ward stadt,0.105508,0.043005,0.159503,0.185363,0.064471,0.109294,0.262478,0.085391,0.082646,0.033027,...,0.000354,0.001765,0.005435,0.006793,0.000688,0.003209,0.003165,0.002525,0.001657,0.000545
scheint presse werde,0.041949,0.066149,0.032659,0.080662,0.057978,0.080329,0.047089,0.217535,0.104552,0.061286,...,0.008846,0.004118,0.002329,0.001359,0.000688,0.003209,0.000633,0.003535,0.000552,0.004902
geschütze januar abgewiesen,0.012288,0.003282,0.000632,0.004452,0.000464,0.002813,0.004605,0.005373,0.004694,0.006888,...,0.737792,0.713529,0.578416,0.637228,0.472490,0.304878,0.263924,0.343939,0.221547,0.245643
ohne seinen wohl,0.218220,0.178411,0.244627,0.248397,0.108071,0.202855,0.180481,0.239941,0.321906,0.211409,...,0.013800,0.034706,0.002329,0.001359,0.010316,0.004493,0.014557,0.000505,0.002762,0.012527
ihr immer ganz,0.308051,0.260622,0.209650,0.224537,0.182282,0.233695,0.285948,0.238569,0.217496,0.281703,...,0.001062,0.000588,0.016304,0.004076,0.003439,0.005777,0.006962,0.003535,0.001657,0.004902
ihnen ihrer worden,0.047034,0.050604,0.072271,0.060007,0.093228,0.088039,0.059863,0.057727,0.064438,0.058460,...,0.002477,0.000588,0.003882,0.009511,0.002063,0.003209,0.006962,0.001515,0.002762,0.002723
südlich juli märz,0.000424,0.000864,0.000211,0.001246,0.004174,0.001771,0.000149,0.001715,0.002703,0.002649,...,0.175867,0.194706,0.306677,0.262228,0.431224,0.648909,0.682911,0.623737,0.746409,0.681373


## 3. Visualization

Now we can see the topics in the model with the following function:

**Hint:** Depending on the number of topics chosen in step 2, you might have to adjust *num_topics* in this step accordingly.

In [33]:
mallet.show_topics_keys("tutorial_supplementals/mallet_output", num_topics=10)

,Key 1,Key 2,Key 3,Key 4,Key 5,Key 6,Key 7,Key 8,Key 9,Key 10
Topic 1,drei,september,deutscher,england,wegen,paris,regierung,deutschland,gebracht,machen
Topic 2,leben,deutschland,kunst,erste,könig,geschichte,professor,verfasser,ganze,kam
Topic 3,vom,einmal,wiener,waren,berliner,zwei,steht,allgemeinen,schreiben,eben
Topic 4,leipzig,ward,stadt,mag,herrn,darauf,möglich,sieht,theil,sollte
Topic 5,scheint,presse,werde,sei,irgend,dadurch,frage,eigentlich,vom,briefe
Topic 6,geschütze,januar,abgewiesen,genommen,östlich,kriegstagebuch,franzosen,angriff,engländer,truppen
Topic 7,ohne,seinen,wohl,dieses,diesem,hatte,lassen,muß,ihren,unserer
Topic 8,ihr,immer,ganz,nun,sondern,ihn,nichts,wurde,keine,weil
Topic 9,ihnen,ihrer,worden,diesen,damit,halten,wurden,frankreich,menschen,französischen
Topic 10,südlich,juli,märz,april,zwischen,stellungen,abgewiesen,nördlich,heftige,gestürmt


### 3.1. Distribution of topics

#### Distribution of topics over all documents

The distribution of topics over all documents can now be visualized in a heat map.

In [34]:
plot = visualization.doc_topic_heatmap_interactive(doc_topics, title="Grenzbote")
from bokeh.io import show
show(plot, notebook_handle=True)

Loading BokehJS ...

#### Distribution of topics in a single documents

To take closer look on the topics in a single text, we can use the follwing function that shows all the topics in a text and their respective proportions. To select the document, we have to give its index to the function.

In [35]:
visualization.plot_doc_topics(doc_topics, 0)

<module 'matplotlib.pyplot' from '/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py'>